In my [last post](/2019/04/simulating-historical-performance-of-leveraged-etfs-in-python/) we discussed simulation of the 3x leveraged S&P 500 ETF, UPRO, and demonstrated why a 100% long UPRO portfolio may not be the best idea. In this post we will analyze the simulated historical performance of another 3x leveraged ETF, TMF, and explore a leveraged variation of Jack Bogle's 60/40 equity/bond allocation.

First lets import the libraries we need.

In [2]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
%matplotlib inline

We'll also need the `sim_leverage` function from my [last post](/2019/04/simulating-historical-performance-of-leveraged-etfs-in-python/).

In [5]:
def sim_leverage(proxy, leverage=1, expense_ratio = 0.0, initial_value=1.0):
    """
    Simulates a leverage ETF given its proxy, leverage, and expense ratio.
    
    Daily percent change is calculated by taking the daily percent change of
    the proxy, subtracting the daily expense ratio, then multiplying by the leverage.
    """
    pct_change = proxy.pct_change(1)
    pct_change = (pct_change - expense_ratio / 252) * leverage
    sim = (1 + pct_change).cumprod() * initial_value
    sim[0] = initial_value
    return sim

Now we can create simulations for both UPRO and TMF using the following values:

| ETF | Expense Ratio | Proxy | Proxy Inception Date |
|-----|---------------|-------|----------------------|
|UPRO | 0.92%         |VFINX  |  08/31/1976          |
|TMF  | 1.09%         |VUSTX  |  05/19/1986          |

We'll use 05/19/1986 as our start date as that is when we'll have data for both proxies.

In [15]:
start = datetime.datetime(1986, 5, 19)
end = datetime.datetime(2019, 1, 1)

vfinx = web.DataReader("VFINX", "yahoo", start, end)["Adj Close"]
vustx = web.DataReader("VUSTX", "yahoo", start, end)["Adj Close"]

upro_sim = sim_leverage(vfinx, leverage=3.0, expense_ratio=0.0092).to_frame("close")
tmf_sim = sim_leverage(vustx, leverage=3.0, expense_ratio=0.0109).to_frame("close")

Before we look at a multi-asset strategy, lets see how each of the assets perform with a simple buy-and-hold strategy. For backtesting our strategies, we will be using [Backtrader](https://www.backtrader.com/), a popular Python backtesting libray that also supports live trading.

In order for our data to work with Backtrader, we will have to fill in the open, high, low, and volume columns. For simplicity we will copy the close price to all columns, sinc

In [16]:
for column in ["open", "high", "low"]:
    upro_sim[column] = upro_sim["close"]
    tmf_sim[column] = tmf_sim["close"]
    
upro_sim["volume"] = 0
tmf_sim["volume"] = 0